# 2021 Championship EDA

In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [137]:
team_stats = pd.read_csv('../data/FS_champ_team_2021.csv')
games = pd.read_csv('../data/FS_champ_games_2021.csv')
goal_timings = pd.read_csv('../data/Goal_timings_champ_2021.csv')

In [138]:
team_stats.describe()

,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,wins_away,draws,draws_home,draws_away,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
count,24.000000,24.000000,24.000000,24.0,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,46.416667,23.208333,23.208333,0.0,17.375000,9.583333,7.791667,11.666667,5.833333,5.833333,...,6.625000,5.541667,9.208333,25.166667,25.000000,25.125000,37.458333,33.708333,41.333333,4.583333
std,0.974308,0.508977,0.508977,0.0,5.427887,3.399702,2.918891,2.632021,2.119612,2.258639,...,2.533472,2.395270,3.833255,5.654292,8.968544,9.728097,11.754663,14.131307,14.187155,3.295803
min,46.000000,23.000000,23.000000,0.0,11.000000,5.000000,4.000000,6.000000,2.000000,2.000000,...,3.000000,1.000000,4.000000,13.000000,9.000000,9.000000,15.000000,8.000000,17.000000,0.000000
25%,46.000000,23.000000,23.000000,0.0,12.750000,7.000000,6.000000,10.000000,4.000000,4.000000,...,4.750000,4.000000,6.000000,22.000000,17.000000,17.000000,30.000000,26.000000,30.000000,2.000000
50%,46.000000,23.000000,23.000000,0.0,16.000000,8.500000,7.500000,11.000000,6.000000,6.000000,...,6.500000,5.000000,9.000000,24.000000,25.500000,25.500000,39.000000,32.500000,38.500000,4.000000
75%,46.000000,23.000000,23.000000,0.0,20.000000,12.000000,9.250000,13.000000,7.000000,8.000000,...,8.250000,7.250000,11.000000,28.000000,30.000000,33.500000,46.000000,40.000000,52.000000,6.250000
max,49.000000,25.000000,25.000000,0.0,29.000000,19.000000,15.000000,17.000000,10.000000,10.000000,...,12.000000,10.000000,17.000000,37.000000,43.000000,43.000000,57.000000,61.000000,70.000000,13.000000


In [139]:
games.describe()

,timestamp,attendance,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,...,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no
count,5.570000e+02,0.0,552.00000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,...,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000
mean,1.610614e+09,NaN,23.50000,1.402819,1.216391,1.489587,1.258833,1.253142,1.052065,2.305206,...,3.004165,2.613393,3.388546,3.425189,1.390557,2.141490,3.845907,7.589318,1.932047,1.793914
std,5.939879e+06,NaN,13.28796,0.675394,0.604858,0.398057,0.353639,1.183905,1.013834,1.491049,...,0.934704,0.862611,0.369592,1.333672,0.083534,0.231239,0.609371,1.478715,0.142656,0.128460
min,1.599850e+09,NaN,1.00000,0.000000,0.000000,0.830000,0.650000,0.000000,0.000000,0.000000,...,0.000000,1.250000,2.900000,1.380000,1.150000,1.500000,2.300000,3.900000,1.540000,1.540000
25%,1.605971e+09,NaN,12.00000,1.000000,0.830000,1.130000,1.000000,0.000000,0.000000,1.000000,...,2.710000,2.000000,3.200000,2.540000,1.330000,2.000000,3.400000,6.700000,1.830000,1.710000
50%,1.610809e+09,NaN,23.50000,1.440000,1.220000,1.430000,1.300000,1.000000,1.000000,2.000000,...,3.140000,2.450000,3.300000,3.100000,1.380000,2.110000,3.800000,7.500000,1.950000,1.770000
75%,1.615638e+09,NaN,35.00000,1.830000,1.570000,1.750000,1.520000,2.000000,2.000000,3.000000,...,3.510000,3.000000,3.500000,4.000000,1.450000,2.300000,4.250000,8.500000,2.000000,1.870000
max,1.622297e+09,NaN,46.00000,3.000000,3.000000,2.570000,2.130000,7.000000,5.000000,9.000000,...,7.000000,8.500000,6.500000,12.000000,1.620000,2.850000,6.500000,16.000000,2.350000,2.300000


In [140]:
games.columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [141]:
games = games[0:552]
games
#Dropping playoff matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1599849900,Sep 11 2020 - 6:45pm,complete,NaN,Watford,Middlesbrough,Keith Stroud,1.0,0.00,0.00,...,2.00,3.45,4.05,1.39,2.20,4.15,8.25,1.95,1.74,Vicarage Road (Watford)
1,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Birmingham City,Brentford,Tony Harrington,1.0,0.00,0.00,...,4.60,3.50,1.87,1.33,2.00,3.65,7.00,1.87,1.80,St Andrew's Trillion Trophy Stadium (Birmingham)
2,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Wycombe Wanderers,Rotherham United,James Linington,1.0,0.00,0.00,...,2.95,3.30,2.50,1.31,1.95,3.50,6.70,1.74,1.95,Adams Park (High Wycombe- Buckinghamshire)
3,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,AFC Bournemouth,Blackburn Rovers,Gavin Ward,1.0,0.00,0.00,...,1.95,3.60,4.05,1.28,1.91,3.25,6.25,1.74,1.95,Vitality Stadium (Bournemouth- Dorset)
4,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Barnsley,Luton Town,Jarred Gillett,1.0,0.00,0.00,...,1.95,3.60,4.00,1.30,1.95,3.40,6.60,1.77,1.91,Oakwell (Barnsley- South Yorkshire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1620473400,May 08 2021 - 11:30am,complete,NaN,Middlesbrough,Wycombe Wanderers,Martin Atkinson,46.0,1.68,0.64,...,1.80,3.70,3.95,1.27,1.82,3.00,5.25,1.74,2.00,Riverside Stadium (Middlesbrough)
548,1620473400,May 08 2021 - 11:30am,complete,NaN,Nottingham Forest,Preston North End,Darren Bond,46.0,1.18,1.45,...,2.23,3.30,3.05,1.32,1.92,3.25,6.00,1.71,2.00,The City Ground (Nottingham- Nottinghamshire)
549,1620473400,May 08 2021 - 11:30am,complete,NaN,Queens Park Rangers,Luton Town,Tony Harrington,46.0,1.55,1.32,...,2.12,3.50,3.10,1.27,1.68,3.00,5.25,1.67,2.10,Kiyan Prince Foundation Stadium (London)
550,1620473400,May 08 2021 - 11:30am,complete,NaN,Reading,Huddersfield Town,Jarred Gillett,46.0,1.77,0.77,...,2.02,3.55,3.30,1.28,1.90,2.80,5.25,1.71,2.00,Madejski Stadium (Reading- Berkshire)


In [142]:
games['referee'].unique()

array(['Keith Stroud', 'Tony Harrington', 'James Linington', 'Gavin Ward',
       'Jarred Gillett', 'John Brooks', 'Andy Madley', 'Darren Bond',
       'Geoff Eltringham', 'Jeremy Simpson', 'Robert Jones',
       'Tim Robinson', 'Steve Martin', 'Peter Bankes', 'Craig Pawson',
       'Dean Whitestone', 'Matt Donohue', 'Leigh Doughty',
       'Oliver Langford', 'Joshua Smith', 'David Webb', 'Andy Woolmer',
       'David Coote', 'Stuart Attwell', 'Graham Scott',
       'Michael Salisbury', 'Andy Davies', 'Simon Hooper', 'Kevin Friend',
       'Darren England', 'Andre Marriner', 'Lee Mason', 'Chris Kavanagh',
       'Jonathan Moss', 'Craig Hicks', 'Thomas Bramall', 'Paul Tierney',
       'John Busby', 'Mike Dean', 'Martin Atkinson'], dtype=object)

In [143]:
refcards = games[['referee', 'home_team_yellow_cards', 'away_team_yellow_cards']]
refcards['total_cards'] = refcards['home_team_yellow_cards'] +  refcards['away_team_yellow_cards']
refcards.head()

<ipython-input-143-a47f93d844e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refcards['total_cards'] = refcards['home_team_yellow_cards'] +  refcards['away_team_yellow_cards']


,referee,home_team_yellow_cards,away_team_yellow_cards,total_cards
0,Keith Stroud,1,0,1
1,Tony Harrington,4,1,5
2,James Linington,1,1,2
3,Gavin Ward,2,0,2
4,Jarred Gillett,1,3,4


# Goal Analysis

In [144]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [145]:
print(f'There were {season_goal_total} goals scored this season')

There were 1274 goals scored this season


In [146]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 569 goals scored in the 1H this season, 


In [147]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.45

In [148]:
games['total_goals_at_half_time'].count()

552

In [149]:
games['total_goals_at_half_time'].value_counts()

1    207
0    190
2    112
3     36
4      5
5      2
Name: total_goals_at_half_time, dtype: int64

In [150]:
(191/557) * 100
#0-0 halftime

34.29084380610413

In [151]:
(190+207) / (557)
# over 70% of games had 1 goal or under in the first half

0.7127468581687613

# Adding more columns

In [152]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [153]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

,common_name,xg_against_avg_overall
0,Stoke City,1.21
1,Middlesbrough,1.13
2,AFC Bournemouth,1.29
3,Swansea City,1.28
4,Watford,1.29


In [154]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [155]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [156]:
fill_stats('Norwich City')
fill_stats('Blackburn Rovers')
fill_stats('Brentford')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Watford')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Rotherham United')
fill_stats('Wycombe Wanderers')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Sheffield Wednesday')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

Let's write a function to help fill xg in our games df

In [157]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go



xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe



fill_stats('Norwich City')
fill_stats('Blackburn Rovers')
fill_stats('Brentford')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Watford')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Rotherham United')
fill_stats('Wycombe Wanderers')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Sheffield Wednesday')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

# Over/Under 2.5?

In [158]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [159]:
games['over/under'].value_counts()

0    326
1    226
Name: over/under, dtype: int64

In [160]:
226/552
#Only 40% were over 2.5

0.40942028985507245

# xG Over/Under 2.5? 

In [161]:
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [162]:
games['xg_over/under'].value_counts()

1    369
0    183
Name: xg_over/under, dtype: int64

In [163]:
369/552

0.6684782608695652

# First Week

In [164]:
first_matchweek = games[0:12]

In [165]:
first_matchweek['over/under'].value_counts()

0    10
1     2
Name: over/under, dtype: int64

In [166]:
first_matchweek

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_btts_no,stadium_name,home_xg,away_xg,home_xg_against,away_xg_against,home_total_corners,away_total_corners,over/under,xg_over/under
0,1599849900,Sep 11 2020 - 6:45pm,complete,NaN,Watford,Middlesbrough,Keith Stroud,1.0,0.0,0.0,...,1.74,Vicarage Road (Watford),1.36,1.44,1.29,1.13,5.61,5.96,0,0
1,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Birmingham City,Brentford,Tony Harrington,1.0,0.0,0.0,...,1.80,St Andrew's Trillion Trophy Stadium (Birmingham),0.94,1.55,1.27,1.12,4.11,4.73,0,0
2,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Wycombe Wanderers,Rotherham United,James Linington,1.0,0.0,0.0,...,1.95,Adams Park (High Wycombe- Buckinghamshire),1.32,1.32,1.59,1.37,4.3,5.04,0,0
3,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,AFC Bournemouth,Blackburn Rovers,Gavin Ward,1.0,0.0,0.0,...,1.95,Vitality Stadium (Bournemouth- Dorset),1.44,1.75,1.29,1.29,5.21,4.61,1,1
4,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Barnsley,Luton Town,Jarred Gillett,1.0,0.0,0.0,...,1.91,Oakwell (Barnsley- South Yorkshire),1.43,1.29,1.08,1.34,4.6,5.13,0,0
5,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Bristol City,Coventry City,John Brooks,1.0,0.0,0.0,...,1.91,Ashton Gate (Bristol),0.81,1.1,1.66,1.36,3.28,4.09,1,1
6,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Cardiff City,Sheffield Wednesday,Andy Madley,1.0,0.0,0.0,...,1.80,Cardiff City Stadium (Cardiff (Caerdydd)),1.35,0.94,1.36,1.4,4.67,4.28,0,0
7,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Derby County,Reading,Darren Bond,1.0,0.0,0.0,...,1.95,Pride Park Stadium (Derby),1.11,1.36,1.24,1.26,4.61,4.24,0,0
8,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Huddersfield Town,Norwich City,Geoff Eltringham,1.0,0.0,0.0,...,1.83,John Smith's Stadium (Huddersfield- West Yorks...,1.11,1.89,1.44,1.34,4.44,4.98,0,1
9,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Millwall,Stoke City,Jeremy Simpson,1.0,0.0,0.0,...,1.67,The Den (London),1.37,1.28,1.34,1.21,4.36,4.76,0,0


# Let's take out the final week now and check percentage

In [167]:
first_41 = games[0:540]
first_41['over/under'].value_counts()

0    323
1    217
Name: over/under, dtype: int64

In [168]:
217/540
# Percentage of games that went over

0.40185185185185185

# When were these goals scored?

Second half stoppage time goals will start at 91 i.e. 90+1 will be 91
First half stoppage time goals will start at 100 i.e. 45+1 = 110 ----

In [169]:
goal_timings['All_goal_timings'].unique()

array(['11', '37', '25', '1', '54', '44', '16', '58', '34', '81', '31',
       '67', '35', '90', '4', '28', '59', '39', "45'2", '41', '33', '46',
       '8', '86', '2', '3', '9', '19', "45'4", '87', '79', '53', '63',
       '62', '13', '64', '12', '83', '70', '27', '30', '66', '24', '22',
       '5', '15', '6', "45'3", '36', '43', '71', '40', '55', '50', '60',
       '84', '56', '85', '78', '69', '48', '77', '20', "45'1", '14', '26',
       '82', '61', "90'2", '68', '10', '52', '80', '21', '47', '7', '88',
       '76', '23', '17', '75', '18', '57', '74', '65', '32', "90'3", '45',
       '42', '72', '38', '49', '51', "90'5", "90'1", "90'4", '89', "90'6",
       '73', '29', "90'8", "90'12", "90'7"], dtype=object)

In [170]:
goal_timings['All_goal_timings'].value_counts()

87       22
27       21
83       21
67       20
76       20
         ..
10        5
90'7      4
45'3      3
90'8      2
90'12     1
Name: All_goal_timings, Length: 103, dtype: int64

Change stoppage time goals so we can cast them as floats

In [171]:
goal_timings = goal_timings.replace(to_replace="90'1", value = "91")
goal_timings = goal_timings.replace(to_replace="90'2", value = "92")
goal_timings = goal_timings.replace(to_replace="90'3", value = "93")
goal_timings = goal_timings.replace(to_replace="90'4", value = "94")
goal_timings = goal_timings.replace(to_replace="90'5", value = "95")
goal_timings = goal_timings.replace(to_replace="90'6", value = "96")
goal_timings = goal_timings.replace(to_replace="90'7", value = "97")
goal_timings = goal_timings.replace(to_replace="90'8", value = "98")
goal_timings = goal_timings.replace(to_replace="90'9", value = "99")
goal_timings = goal_timings.replace(to_replace="90'10", value = "100")
goal_timings = goal_timings.replace(to_replace="90'11", value = "101")
goal_timings = goal_timings.replace(to_replace="90'12", value = "102")
goal_timings = goal_timings.replace(to_replace="45'1", value = "110")
goal_timings = goal_timings.replace(to_replace="45'2", value = "111")
goal_timings = goal_timings.replace(to_replace="45'3", value = "112")
goal_timings = goal_timings.replace(to_replace="45'4", value = "113")

In [172]:
goal_timings['All_goal_timings'] = goal_timings['All_goal_timings'].astype('int')

In [173]:
timing_totals = goal_timings['All_goal_timings'].value_counts()
timing_totals = pd.DataFrame(timing_totals)

In [174]:
timing_totals = timing_totals.rename_axis("minute", axis="columns")
timing_totals = timing_totals.rename(columns={"All_goal_timings":"count"})

In [175]:
timing_totals['section'] = ""
timing_totals.head()

minute,count,section
87,22,
27,21,
83,21,
76,20,
85,20,


# In how many games did a team have more xG but lose?